## Importing all the required packages

In [1]:
import numpy as np
import pandas as pd
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [2]:
# Importing the dataset
df = pd.read_csv('/Users/andre/Documents/GitHub/immo-analysis-project/data/dataset_immo_.csv')

## Dataset cleaning

In [3]:
# Renaming some columns names in order to avoid problems

df.rename(columns={'swimming-pool': 'swimming_pool'}, inplace=True)
df.rename(columns={'state-building': 'state_building'}, inplace=True)
df.rename(columns={'land-surface': 'land_surface'}, inplace=True)

In [4]:
# Removing duplicated rows
df.duplicated()
# Replacing NaN with 0
df.fillna(0, inplace=True)

In [5]:
# Removing all the rows where there's no information on locality
df.drop(df[df['locality'] == 0].index, inplace=True)
# Removing the columns type-transaction and url as it is not relevant
df.drop(columns=['type-transaction'], inplace=True)
df.drop(columns=['url'], inplace=True)
# Removing the columns area garden and terrace
df.drop(columns=['area_terrace'], inplace=True)
df.drop(columns=['area-garden'], inplace=True)
# Removing the column number of facades
df.drop(columns=['n-facades'], inplace=True)

In [6]:
# Removing outliers 
def remove_outliers(df,columns,n_std):
    for col in columns:
        print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

df = remove_outliers(df, ['price', 'living_area'], 3)

df

Working on column: price
Working on column: living_area


,locality,Type_property,subtype_property,price,n_rooms,living_area,equipped_kitchen,furnished,fireplace,terrace,garden,land_surface,swimming_pool,state_building
0,Borgerhout,APARTMENT,APARTMENT,245000.0,2.0,106.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,JUST_RENOVATED
1,Leuven,APARTMENT,FLAT_STUDIO,199000.0,0.0,31.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,GOOD
2,Lede,APARTMENT,APARTMENT,195000.0,2.0,91.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,AS_NEW
3,Gent,APARTMENT,APARTMENT,185000.0,2.0,101.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,TO_RESTORE
4,Hotton,APARTMENT,PENTHOUSE,375000.0,3.0,214.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,TO_BE_DONE_UP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10014,Brugge,HOUSE,HOUSE,499000.0,3.0,174.0,1.0,0.0,0.0,1.0,1.0,1290.0,0.0,GOOD
10015,Aalst,HOUSE,VILLA,495000.0,5.0,238.0,1.0,0.0,0.0,1.0,1.0,1200.0,0.0,AS_NEW
10016,Waregem,HOUSE,HOUSE,500000.0,2.0,140.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,TO_BE_DONE_UP
10018,Blankenberge,HOUSE,VILLA,525000.0,5.0,290.0,1.0,0.0,0.0,1.0,1.0,737.0,0.0,GOOD


## Preprocessing the dataset for the model building

In [7]:
# Standardizing the values of the columns: number of rooms, living area and land surface

from sklearn.preprocessing import scale

# Subset of columns to transform
cols = ['n_rooms','living_area', 'land_surface']

# Overwrite old columns with transformed columns
df[cols] = scale(df[cols])

df

,locality,Type_property,subtype_property,price,n_rooms,living_area,equipped_kitchen,furnished,fireplace,terrace,garden,land_surface,swimming_pool,state_building
0,Borgerhout,APARTMENT,APARTMENT,245000.0,-0.594528,-0.530233,1.0,0.0,1.0,1.0,0.0,-0.248029,0.0,JUST_RENOVATED
1,Leuven,APARTMENT,FLAT_STUDIO,199000.0,-1.931874,-1.332928,1.0,1.0,1.0,1.0,0.0,-0.248029,0.0,GOOD
2,Lede,APARTMENT,APARTMENT,195000.0,-0.594528,-0.690772,1.0,0.0,0.0,1.0,0.0,-0.248029,0.0,AS_NEW
3,Gent,APARTMENT,APARTMENT,185000.0,-0.594528,-0.583746,1.0,0.0,1.0,1.0,0.0,-0.248029,0.0,TO_RESTORE
4,Hotton,APARTMENT,PENTHOUSE,375000.0,0.074145,0.625648,1.0,0.0,0.0,0.0,0.0,-0.248029,0.0,TO_BE_DONE_UP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10014,Brugge,HOUSE,HOUSE,499000.0,0.074145,0.197544,1.0,0.0,0.0,1.0,1.0,0.439874,0.0,GOOD
10015,Aalst,HOUSE,VILLA,495000.0,1.411490,0.882510,1.0,0.0,0.0,1.0,1.0,0.391881,0.0,AS_NEW
10016,Waregem,HOUSE,HOUSE,500000.0,-0.594528,-0.166345,1.0,0.0,1.0,1.0,0.0,-0.248029,0.0,TO_BE_DONE_UP
10018,Blankenberge,HOUSE,VILLA,525000.0,1.411490,1.439046,1.0,0.0,0.0,1.0,1.0,0.144982,0.0,GOOD


In [7]:
# Using the function get dummies to transform categorical values

x = df[['locality', 'Type_property', 'subtype_property', 'price', 'n_rooms', 'living_area', 'equipped_kitchen', 
       'furnished', 'fireplace', 'terrace', 'garden', 'land_surface', 'swimming_pool', 'state_building']]

x = pd.get_dummies(data=x, drop_first=True)

x

,price,n_rooms,living_area,equipped_kitchen,furnished,fireplace,terrace,garden,land_surface,swimming_pool,...,subtype_property_TOWN_HOUSE,subtype_property_TRIPLEX,subtype_property_VILLA,state_building_0,state_building_AS_NEW,state_building_GOOD,state_building_JUST_RENOVATED,state_building_TO_BE_DONE_UP,state_building_TO_RENOVATE,state_building_TO_RESTORE
0,245000.0,2.0,106.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,True,False,False,False
1,199000.0,0.0,31.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
2,195000.0,2.0,91.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,False,False,False,False,True,False,False,False,False,False
3,185000.0,2.0,101.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
4,375000.0,3.0,214.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10014,499000.0,3.0,174.0,1.0,0.0,0.0,1.0,1.0,1290.0,0.0,...,False,False,False,False,False,True,False,False,False,False
10015,495000.0,5.0,238.0,1.0,0.0,0.0,1.0,1.0,1200.0,0.0,...,False,False,True,False,True,False,False,False,False,False
10016,500000.0,2.0,140.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False
10018,525000.0,5.0,290.0,1.0,0.0,0.0,1.0,1.0,737.0,0.0,...,False,False,True,False,False,True,False,False,False,False


## Model building - XGBoost

In [12]:
# Pipeline building

# Assigning X and y

X = x.drop(['price'], axis=1).to_numpy()
y = x['price'].to_numpy().reshape(-1, 1)

# Splitting the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanting the model to an object and trainning it

regressor = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = 123)

regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [13]:
# Checking up the model score for the trainning set
print('Training set score: {:.2f}'.format(regressor.score(X_train, y_train)))

# Checking up the model score for the test set

print('Test set score: {:.2f}'.format(regressor.score(X_test, y_test)))

# Calculating the RMSE

from sklearn.metrics import mean_squared_error as MSE

pred = regressor.predict(X_test)

rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

Training set score: 0.74
Test set score: 0.63
RMSE :  203364.847907
